In [1]:
import requests
import pandas as pd

# USGS 地震数据的 GeoJSON URL（过去 30 天，所有地震）
url = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson"

# 获取数据
response = requests.get(url)
data = response.json()

# 将地震数据转换为 DataFrame
earthquakes = pd.json_normalize(data['features'])

# 提取所需字段
earthquakes = earthquakes[['properties.place', 'properties.mag', 'geometry.coordinates']]
earthquakes.columns = ['Location', 'Magnitude', 'Coordinates']

# 分离经度和纬度
earthquakes['Longitude'] = earthquakes['Coordinates'].apply(lambda x: x[0])
earthquakes['Latitude'] = earthquakes['Coordinates'].apply(lambda x: x[1])

# 删除无用列
earthquakes.drop(columns=['Coordinates'], inplace=True)

# 显示前几行数据
print(earthquakes.head())

                         Location  Magnitude  Longitude  Latitude
0  14 km ENE of Yerington, Nevada        1.2  -119.0194   39.0458
1  14 km ENE of Yerington, Nevada        1.3  -119.0222   39.0549
2  13 km ENE of Yerington, Nevada        1.1  -119.0235   39.0475
3  14 km ENE of Yerington, Nevada        3.1  -119.0105   39.0494
4   43 km NW of Ninilchik, Alaska        1.8  -152.1739   60.3438


In [3]:
import folium

# 创建地图对象，初始位置设置为全球视图
m = folium.Map(location=[0, 0], zoom_start=2)

# 根据地震数据添加标记
for index, row in earthquakes.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],        radius=row['Magnitude'] * 2,  # 震级决定圆的大小
        color='crimson',
        fill=True,
        fill_color='crimson',
        fill_opacity=0.6,
        popup=f"Location: {row['Location']}\nMagnitude: {row['Magnitude']}"
    ).add_to(m)

# 保存地图为 HTML 文件
m.save('earthquake_map.html')

# 在 Jupyter Notebook 中显示地图（如果使用 Jupyter Notebook）
# m

In [6]:
import requests
import pandas as pd
import folium

# Step 1: Fetch earthquake data from USGS GeoJSON API
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson'
response = requests.get(url)
data = response.json()

# Step 2: Extract relevant fields
earthquake_list = []
for feature in data['features']:
    properties = feature['properties']
    geometry = feature['geometry']
    
    if geometry['type'] == 'Point':
        coords = geometry['coordinates']
        earthquake_list.append({
            'Location': properties['place'],
            'Magnitude': properties['mag'],
            'Longitude': coords[0],
            'Latitude': coords[1],
            'Time': pd.to_datetime(properties['time'], unit='ms')
        })

# Step 3: Convert to a DataFrame
earthquakes = pd.DataFrame(earthquake_list)

# Step 4: Define a function to assign color based on magnitude
def get_color(mag):
    if mag < 3.0:
        return 'green'
    elif 3.0 <= mag < 5.0:
        return 'orange'
    else:
        return 'red'

# Step 5: Create base map
earthquake_map = folium.Map(location=[0, 0], zoom_start=2)#, tiles="CartoDB positron"

# Step 6: Add enhanced markers
for _, row in earthquakes.iterrows():
    mag = row['Magnitude']
    if pd.notnull(mag):
        color = get_color(mag)
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=max(3, mag * 2),
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(
                f"<strong>Location:</strong> {row['Location']}<br>"
                f"<strong>Magnitude:</strong> {mag:.2f}<br>"
                f"<strong>Time:</strong> {row['Time']}<br>"
                f"<strong>Coordinates:</strong> ({row['Latitude']:.2f}, {row['Longitude']:.2f})",
                max_width=300
            )
        ).add_to(earthquake_map)

# Step 7: Add a legend (as a custom HTML element)
legend_html = '''
<div style="position: fixed; 
     bottom: 30px; left: 30px; width: 180px; height: 130px; 
     background-color: white; z-index:9999; font-size:14px;
     border:2px solid grey; padding: 10px;">
<b>Magnitude Legend</b><br>
<span style="color:green;">●</span> Magnitude < 3.0<br>
<span style="color:orange;">●</span> 3.0 ≤ Magnitude < 5.0<br>
<span style="color:red;">●</span> Magnitude ≥ 5.0
</div>
'''
earthquake_map.get_root().html.add_child(folium.Element(legend_html))

# Step 8: Save the map
earthquake_map.save("enhanced_earthquake_map.html")
print("Enhanced earthquake map saved as 'enhanced_earthquake_map.html'")

Enhanced earthquake map saved as 'enhanced_earthquake_map.html'


In [7]:
import requests
import pandas as pd
import folium
from folium import plugins
from branca.colormap import linear

# Step 1: Fetch earthquake data from USGS GeoJSON API
url = 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_month.geojson'
response = requests.get(url)
data = response.json()

# Step 2: Extract relevant fields
earthquake_list = []
for feature in data['features']:
    properties = feature['properties']
    geometry = feature['geometry']
    
    if geometry['type'] == 'Point':
        coords = geometry['coordinates']
        earthquake_list.append({
            'Location': properties['place'],
            'Magnitude': properties['mag'],
            'Longitude': coords[0],
            'Latitude': coords[1],
            'Time': pd.to_datetime(properties['time'], unit='ms')
        })

# Step 3: Convert to a DataFrame
earthquakes = pd.DataFrame(earthquake_list)

# Step 4: Define a color scale for magnitude using LinearColormap
colormap = linear.YlOrRd_09.scale(0, 9)  # Using a color scale from yellow to red

# Step 5: Create base map
earthquake_map = folium.Map(location=[0, 0], zoom_start=2)

# Step 6: Add markers with color based on magnitude using the colormap
for _, row in earthquakes.iterrows():
    mag = row['Magnitude']
    if pd.notnull(mag):
        color = colormap(mag)  # Get color based on magnitude
        folium.CircleMarker(
            location=[row['Latitude'], row['Longitude']],
            radius=max(3, mag * 2),
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7,
            popup=folium.Popup(
                f"<strong>Location:</strong> {row['Location']}<br>"
                f"<strong>Magnitude:</strong> {mag:.2f}<br>"
                f"<strong>Time:</strong> {row['Time']}<br>"
                f"<strong>Coordinates:</strong> ({row['Latitude']:.2f}, {row['Longitude']:.2f})",
                max_width=300
            )
        ).add_to(earthquake_map)

# Step 7: Add a legend for the colormap (gradient scale)
colormap.add_to(earthquake_map)

# Step 8: Save the map
earthquake_map.save("gradual_earthquake_map.html")
print("Earthquake map with gradient color saved as 'gradual_earthquake_map.html'")

Earthquake map with gradient color saved as 'gradual_earthquake_map.html'
